# 0 Setup

In [1]:
# importing all required packages at the start of the notebook
import IPython
import os
import pandas as pd
import qiime2 as q2
from qiime2 import Visualization
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
os.getcwd() #Get the working directory

'/home/jovyan/assignments/FunGut-Project'

In [3]:
data_dir = "project_data" #Store the folder's path

# 1 Importing the data

In [7]:
%%bash -s $data_dir
mkdir -p "$1"

wget -nc --progress=dot:giga -P "$1" https://polybox.ethz.ch/index.php/s/uV06vmm96ZzB5eM/download/fungut_forward_reads.qza
wget -nc --progress=dot:giga -P "$1" https://polybox.ethz.ch/index.php/s/CA76kKFC9FApqpR/download/fungut_metadata.tsv

chmod -R +rxw "$1"

File ‘project_data/fungut_forward_reads.qza’ already there; not retrieving.

File ‘project_data/fungut_metadata.tsv’ already there; not retrieving.



In [9]:
# To check that our files are in the right place:
qza_file = f"{data_dir}/fungut_forward_reads.qza" #Store the sequences file
tsv_file = f"{data_dir}/fungut_metadata.tsv" #Store the sample metadata file
print("File exists?", os.path.exists(qza_file), os.path.exists(tsv_file))

File exists? True True


# 2 Feature table construction

## 2.1 First overview of our sample and quality score assessment 

In [12]:
!qiime demux summarize \
  --i-data $data_dir/fungut_forward_reads.qza \
  --o-visualization $data_dir/demux-summary.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: project_data/demux-summary.qzv


In [5]:
Visualization.load(f"{data_dir}/demux-summary.qzv")

<visualization: Visualization uuid: e1dc45d5-1a96-457c-a372-eda44cd9f4dd>

The mean length of our reads is 151 nts. We can see that the quality of our reads stays quite high, even at the end of the sequences (mean quality score ~38 at the position 151).

In [10]:
!qiime cutadapt trim-single \
  --i-demultiplexed-sequences $data_dir/fungut_forward_reads.qza \
  --p-front "CTTGGTCATTTAGAGGAAGTAA" \
  --p-adapter "GCATCGATGAAGAACGCAGC" \
  --p-error-rate 0.1 \
  --p-minimum-length 50 \
  --o-trimmed-sequences $data_dir/fungut_forward_reads_trimmed.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved SampleData[SequencesWithQuality] to: project_data/fungut_forward_reads_trimmed.qza


In [12]:
!qiime demux summarize \
  --i-data $data_dir/fungut_forward_reads_trimmed.qza \
  --o-visualization $data_dir/demux-summary_trimmed.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: project_data/demux-summary_trimmed.qzv


In [4]:
Visualization.load(f"{data_dir}/demux-summary_trimmed.qzv")

<visualization: Visualization uuid: 37563b24-1abb-4588-90d0-04629b9f702f>

## 2.2 Denoizing and creation of ASVs

In [23]:
! qiime dada2 denoise-single \
    --i-demultiplexed-seqs $data_dir/fungut_forward_reads.qza \
    --p-trunc-len 0 \
    --p-n-threads 3 \
    --p-max-ee 4 \
    --p-min-fold-parent-over-abundance 4 \
    --o-table $data_dir/dada2_table_no_trunc.qza \
    --o-representative-sequences $data_dir/dada2_rep_set_no_trunc.qza \
    --o-denoising-stats $data_dir/dada2_stats_no_trunc.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: project_data/dada2_table_no_trunc.qza
Saved FeatureData[Sequence] to: project_data/dada2_rep_set_no_trunc.qza
Saved SampleData[DADA2Stats] to: project_data/dada2_stats_no_trunc.qza


In [8]:
# Same thing but with trimmed

! qiime dada2 denoise-single \
    --i-demultiplexed-seqs $data_dir/fungut_forward_reads_trimmed.qza \
    --p-trunc-len 0 \
    --p-n-threads 3 \
    --p-max-ee 4 \
    --p-min-fold-parent-over-abundance 4 \
    --o-table $data_dir/dada2_trimmed_table_no_trunc.qza \
    --o-representative-sequences $data_dir/dada2_trimmed_rep_set_no_trunc.qza \
    --o-denoising-stats $data_dir/dada2_trimmed_stats_no_trunc.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: project_data/dada2_trimmed_table_no_trunc.qza
Saved FeatureData[Sequence] to: project_data/dada2_trimmed_rep_set_no_trunc.qza
Saved SampleData[DADA2Stats] to: project_data/dada2_trimmed_stats_no_trunc.qza


First I tried to put 150 nts as trunc-len, but this disgarded too much sequences. I don't want to put the truncating lenght lower, because we will loose a lot of information on the sequences. As ITS have usually very different length, and that the mean quality of our reads was good, I want to keep all of them. However, after denoizing I want to do a step in order to remove the sequences that are too rare and the ones that come up in too few samples, in order to avoid too much noise in further downstream analysis:

In [24]:
! qiime feature-table filter-features \
  --i-table $data_dir/dada2_table_no_trunc.qza \
  --p-min-frequency 5 \
  --p-min-samples 2 \
  --o-filtered-table $data_dir/dada2_table.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: project_data/dada2_table.qza


In [9]:
# Same thing but with trimmed

! qiime feature-table filter-features \
  --i-table $data_dir/dada2_trimmed_table_no_trunc.qza \
  --p-min-frequency 5 \
  --p-min-samples 2 \
  --o-filtered-table $data_dir/dada2_trimmed_table.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: project_data/dada2_trimmed_table.qza


Now we have filtered all features that have a frequencies smaller than 10% across all samples, as well as features present in only one sample. Now that it's done, I will update the list of sequences so they match.

In [25]:
! qiime feature-table filter-seqs \
  --i-data $data_dir/dada2_rep_set_no_trunc.qza \
  --i-table $data_dir/dada2_table.qza \
  --o-filtered-data $data_dir/dada2_rep_set.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureData[Sequence] to: project_data/dada2_rep_set.qza


In [12]:
# Same thing but with trimmed

! qiime feature-table filter-seqs \
  --i-data $data_dir/dada2_trimmed_rep_set_no_trunc.qza \
  --i-table $data_dir/dada2_trimmed_table.qza \
  --o-filtered-data $data_dir/dada2_trimmed_rep_set.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureData[Sequence] to: project_data/dada2_trimmed_rep_set.qza


Now that our denoizing and filtering is done, we can look at what the result it. Let's start with the denoizing statistics (so here nothing is filtered yet):

In [26]:
! qiime metadata tabulate \
    --m-input-file $data_dir/dada2_stats_no_trunc.qza \
    --o-visualization $data_dir/dada2_stats_no_trunc.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: project_data/dada2_stats_no_trunc.qzv


In [6]:
Visualization.load(f"{data_dir}/dada2_stats_no_trunc.qzv")

<visualization: Visualization uuid: ef3b1e72-96b7-46b3-82e5-2fd959e09d0a>

Eyeballing it the statistics look good, we don't loose to many sequences.

Now we are going to visualize the sequences:

In [28]:
! qiime feature-table tabulate-seqs \
    --i-data $data_dir/dada2_rep_set.qza \
    --o-visualization $data_dir/dada2_rep_set.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: project_data/dada2_rep_set.qzv


In [13]:
# Same but with trimmed

! qiime feature-table tabulate-seqs \
    --i-data $data_dir/dada2_trimmed_rep_set.qza \
    --o-visualization $data_dir/dada2_trimmed_rep_set.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: project_data/dada2_trimmed_rep_set.qzv


In [7]:
Visualization.load(f"{data_dir}/dada2_rep_set.qzv")

<visualization: Visualization uuid: 7e308edc-ffc7-4599-991d-e0fed6d52d3f>

In [14]:
# Same but with trimmed

Visualization.load(f"{data_dir}/dada2_trimmed_rep_set.qzv")

<visualization: Visualization uuid: 301c0df3-ce3d-4aa1-8dfc-2fc8ab19915e>

We can see that we have 149 unique features (=ASVs) after filtering.

Finally, we are going to create a feature table with the information of our sequences and the sample metadata:

In [33]:
ls project_data

dada2_rep_set.qza           dada2_stats_no_trunc.qzv  fungut_forward_reads.qza*
dada2_rep_set.qzv           dada2_table.qza           fungut_metadata.tsv*
dada2_rep_set_no_trunc.qza  dada2_table_no_trunc.qza
dada2_stats_no_trunc.qza    demux-summary.qzv*


In [34]:
! qiime feature-table summarize \
    --i-table $data_dir/dada2_table.qza \
    --m-sample-metadata-file $data_dir/fungut_metadata.tsv \
    --o-visualization $data_dir/dada2_table.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: project_data/dada2_table.qzv


In [15]:
Visualization.load(f"{data_dir}/dada2_table.qzv")

<visualization: Visualization uuid: 9951df15-db7f-42a8-9a9a-0426acab4455>

Bonus: We can compare this table that we just made that contains the information of the filtered features (with >=10% of frequency and presence in >= 2 samples) with a table created with all the sequences prior to filtering:

In [38]:
! qiime feature-table summarize \
    --i-table $data_dir/dada2_table_no_trunc.qza \
    --m-sample-metadata-file $data_dir/fungut_metadata.tsv \
    --o-visualization $data_dir/dada2_table_no_trunc.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: project_data/dada2_table_no_trunc.qzv


In [39]:
Visualization.load(f"{data_dir}/dada2_table_no_trunc.qzv")

<visualization: Visualization uuid: 5370332e-ceb1-4ad5-a087-691c38e7f87b>

Thing is that with my filter I did remove a lot of special features that seemed either to appear in only one sample, or to be at a less than 10% frequency. I don't know how to decide if that's good or not.